This was wrtten is Google Colab so may need modification in places file path

In [1]:
from google.colab import files
uploaded=files.upload()

Saving kaggle.json to kaggle.json


In [4]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d rounakbanik/the-movies-dataset

the-movies-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [41]:
pip install surprise

     |████████████████████████████████| 11.8 MB 4.5 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1619393 sha256=146ca85cd99faa6b23fc9a5a389e0143f934b15bb42eabb1288447c04bcacadd
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [2]:
import os 
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from zipfile import ZipFile
from wordcloud import WordCloud,STOPWORDS
import ast

In [3]:

import ast
import nltk
import unicodedata
import re
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

#DataPreprecessing

unzip multiple csv file in 'the-movies-dataset.zip' to 'temp_csv' file .

In [5]:
with ZipFile('/content/the-movies-dataset.zip', 'r') as zipObj:
   # Get a list of all archived file names from the zip
   listOfFileNames = zipObj.namelist()
   # Iterate over the file names
   for fileName in listOfFileNames:
       # Check filename endswith csv
       if fileName.endswith('.csv'):
           # Extract a single file from zip
           zipObj.extract(fileName, 'temp_csv')

read each of the csv file in 'temp_csv' to separate variable.

In [6]:
credits=pd.read_csv('/content/temp_csv/credits.csv',header=0,sep=',',quotechar='"')
keywords=pd.read_csv('/content/temp_csv/keywords.csv',header=0,sep=',',quotechar='"')
links=pd.read_csv('/content/temp_csv/links.csv')
links_small=pd.read_csv('/content/temp_csv/links_small.csv',header=0,sep=',',quotechar='"')
movies=pd.read_csv('/content/temp_csv/movies_metadata.csv',header=0,sep=',',quotechar='"')
df_ratings_all=pd.read_csv('/content/temp_csv/ratings.csv',header=0,sep=',',quotechar='"')
ratings_small=pd.read_csv('/content/temp_csv/ratings_small.csv',header=0,sep=',',quotechar='"')

In [7]:
movies=movies.drop([19730, 29503, 35587])
ratings_small=ratings_small.drop([19730, 29503, 35587])

In [13]:



keywords['id'] = keywords['id'].astype('str')
credits['id'] = credits['id'].astype('str')
df_merge = (credits.merge(movies,on='id')).merge(keywords,on='id')
df = df_merge[:10000]
#df=df_merge
#df.dtypes



In [14]:
#df_merge.head()

In [15]:
# Use certain columns
df = df[['title','cast', 'crew', 'genres','keywords','original_language']]
print(sum(df.isnull().any()))
df.dropna(inplace=True)
print(sum(df.isnull().any()))

0
0


In [16]:
# Format words within the dataset
def clean_data(value):
    value =  unicodedata.normalize('NFD', value).encode('ascii', 'ignore').decode('ascii')
    x = str.lower(re.sub(r"[^a-zA-Z0-9]+", '_', value.strip().replace(" ","_")))
    if(x != '_'):
        return x 
    else:
        return " " 

In [17]:
# And also we want to use words. So we must rearrange these data. For example cast values are like that:
#df.cast

In [18]:
df['cast'] = df['cast'].apply(lambda s: list(ast.literal_eval(s)))
df['cast'] = df['cast'].apply((lambda cast : [clean_data(actor['name']) for actor in cast]))


In [19]:
df['title'] = df['title'].apply((lambda title: clean_data(title)))

In [20]:
df['genres'] = df['genres'].apply(lambda s: list(ast.literal_eval(s)))
df['genres'] = df['genres'].apply((lambda genres : [clean_data(genre['name']) for genre in genres]))

In [21]:
#print(df.keywords)
df['keywords'] = df['keywords'].apply(lambda s: list(ast.literal_eval(s)))
df['keywords'] = df['keywords'].apply((lambda keywords : [clean_data(key['name']) for key in keywords[:5]]))
#df.keywords

In [22]:
# The crew members that key main attract audances to watch a movie should be include in the crew list rest removed https://www.wrapbook.com/blog/film-crew-positions key people to key on film
df['crew'] = df['crew'].apply(lambda s: list(ast.literal_eval(s)))


def get_dir_prod_writer(x):
    names=[]
    for i in x:
        if i['job']=='Director':
            names.append(i['name'])
        elif i['job']=='Producer':
            names.append(i['name'])
        elif i['job'] =='Screenplay':
           names.append(i['name'])   
        elif i['job'] =='Executive Producer':
           names.append(i['name'])              
    return names

df['crew']=df['crew'].apply(get_dir_prod_writer)

0       [{'credit_id': '52fe4284c3a36847f8024f49', 'de...
1       [{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...
2       [{'credit_id': '52fe466a9251416c75077a89', 'de...
3       [{'credit_id': '52fe44779251416c91011acb', 'de...
4       [{'credit_id': '52fe44959251416c75039ed7', 'de...
                              ...                        
9995    [{'credit_id': '52fe467cc3a36847f81019f9', 'de...
9996    [{'credit_id': '59665828c3a36825e8116b9a', 'de...
9997    [{'credit_id': '52fe481ac3a368484e0e94f9', 'de...
9998    [{'credit_id': '52fe46349251416c9104cb19', 'de...
9999    [{'credit_id': '52fe4518c3a368484e04830d', 'de...
Name: crew, Length: 10000, dtype: object


In [23]:
#print(df.original_language)
# Dont change anything here
#df.title

In [24]:
# create a bag of word from previous attributes 
df['bag_of_words'] = df['cast'] + df['keywords'] + df['genres'] + df['crew'] 
df['bag_of_words'] = df['bag_of_words'].apply(lambda x: ' '.join(x)) + " " + df['title']

In [25]:
df.head()

,title,cast,crew,genres,keywords,original_language,bag_of_words
0,toy_story,"[tom_hanks, tim_allen, don_rickles, jim_varney...","[John Lasseter, Joss Whedon, Andrew Stanton, J...","[animation, comedy, family]","[jealousy, toy, boy, friendship, friends]",en,tom_hanks tim_allen don_rickles jim_varney wal...
1,jumanji,"[robin_williams, jonathan_hyde, kirsten_dunst,...","[Larry J. Franco, Jonathan Hensleigh, Joe John...","[adventure, fantasy, family]","[board_game, disappearance, based_on_children_...",en,robin_williams jonathan_hyde kirsten_dunst bra...
2,grumpier_old_men,"[walter_matthau, jack_lemmon, ann_margret, sop...",[Howard Deutch],"[romance, comedy]","[fishing, best_friend, duringcreditsstinger, o...",en,walter_matthau jack_lemmon ann_margret sophia_...
3,waiting_to_exhale,"[whitney_houston, angela_bassett, loretta_devi...","[Forest Whitaker, Ronald Bass, Ronald Bass, Ez...","[comedy, drama, romance]","[based_on_novel, interracial_relationship, sin...",en,whitney_houston angela_bassett loretta_devine ...
4,father_of_the_bride_part_ii,"[steve_martin, diane_keaton, martin_short, kim...","[Nancy Meyers, Nancy Meyers, Albert Hackett, C...",[comedy],"[baby, midlife_crisis, confidence, aging, daug...",en,steve_martin diane_keaton martin_short kimberl...


In [26]:
df_ratings_all['movieId'] = df_ratings_all['movieId'].astype('str')
df_ratings_merge = df_ratings_all.merge(movies[['title','id']], left_on='movieId', right_on='id')
df_ratings_merge = df_ratings_merge[['userId','movieId', 'rating', 'title']]
df_ratings_merge


,userId,movieId,rating,title
0,1,110,1.0,Three Colors: Red
1,11,110,3.5,Three Colors: Red
2,22,110,5.0,Three Colors: Red
3,24,110,5.0,Three Colors: Red
4,29,110,3.0,Three Colors: Red
...,...,...,...,...
11437632,270123,140222,3.0,Love and Honor
11437633,270123,151509,3.0,Trapped in the Closet: Chapters 23-33
11437634,270123,159109,1.5,The Rambler
11437635,270694,165159,4.0,Agenzia Riccardo Finzi... praticamente detective


In [27]:
df_ratings_merge_with_different_title = df_ratings_merge
df_ratings_merge_with_different_title['title'] = df_ratings_merge_with_different_title['title'].apply(lambda x: clean_data(x))

In [48]:
df_ratings_merge_with_different_title.head()

,userId,movieId,rating,title
0,1,110,1.0,three_colors_red
1,11,110,3.5,three_colors_red
2,22,110,5.0,three_colors_red
3,24,110,5.0,three_colors_red
4,29,110,3.0,three_colors_red


In [28]:
df_ratings_merge_with_different_title=df_ratings_merge_with_different_title[(df_ratings_merge_with_different_title['rating']>3.5)]
df_ratings_merge_with_different_title.reset_index(drop=True, inplace=True)

In [62]:
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df.bag_of_words)

In [30]:
# Compute cosine similarity between all movie-descriptions I tried with euclidean but gave poor results
similarity = cosine_similarity(tfidf_matrix)
# Remove self-similarity from matrix
similarity -= np.eye(similarity.shape[0])

In [31]:
def get_content_based_recommendations(title):
 
    index = df.reset_index(drop=True)[df.title == clean_data(title)].index
    if len(index) > 0:
        index = index[0]
        n_plot = 10
      
        similar_movies_index = np.argsort(similarity[index])[::-1][:n_plot]
        similar_movies_score = np.sort(similarity[index])[::-1][:n_plot]

        # Get similar movies
        similar_movie_titles = df.iloc[similar_movies_index].index
        return [df.iloc[index].title for index in similar_movies_index]
    else:
        return None

In [33]:
def get_content_based_recommendation_for_user(userId):
    # Check if user id exist
    # if not exist show some of populars
    
    # Get highest ranking score of this user
    highest_ranked_movies = df_ratings_merge[df_ratings_merge['userId'] == userId]
    highest_ranked_movies.sort_values(by=['rating'], ascending=False)
    #print(highest_ranked_movies)
    similar_movies = set()
    for i in range(len(highest_ranked_movies)):
        current_similar_movies = get_content_based_recommendations(highest_ranked_movies.iloc[i].title)
        #This will be define after using Suprise
        if current_similar_movies == None:
            continue
        for movie_title in current_similar_movies:
            similar_movies.add(movie_title)
    
    result_df = pd.DataFrame(similar_movies, columns=['title'])
    
    without_duplicate = df_ratings_merge_with_different_title.drop_duplicates(subset='title', keep='first', inplace = False)    
    result_df = result_df.merge(without_duplicate, on = 'title')
    
    return result_df



In [44]:
df_ratings = df_ratings_all[:1000000]
df_ratings.sample(10)
df_train = df_ratings.sample(frac=0.8)
df_test = df_ratings.drop(df_train.index)
print("Train data: {}, test data: {}".format(len(df_train), len(df_test)))

Train data: 800000, test data: 200000


In [54]:
# import most of Algorithms for valulation did not inculde SVDpp as process time took too long 
from surprise import Reader, Dataset,  SVD, SVDpp, SlopeOne, NMF, NormalPredictor, KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore, BaselineOnly, CoClustering
from surprise.model_selection import cross_validate
# Define the format
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(df_ratings[['userId', 'movieId', 'rating']], reader)


In [56]:
benchmark = []
# Iterate over all algorithms will take time
for algorithm in [SVD(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]:
    # Perform cross validation
    results = cross_validate(algorithm, data, measures=['RMSE'], cv=3, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')  

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...


,test_rmse,fit_time,test_time
Algorithm,,,
SVD,0.844549,35.708838,3.897160
KNNBaseline,0.856529,34.863321,177.876389
BaselineOnly,0.862672,4.793888,4.298427
SlopeOne,0.865837,41.586344,122.078795
KNNWithZScore,0.878604,31.637090,170.269400
KNNWithMeans,0.879508,31.867290,175.963108
NMF,0.881415,46.969916,3.748634
KNNBasic,0.907495,30.372117,168.997492
CoClustering,0.910004,20.238074,4.043315


In [57]:
benchmark = []
# Select the algorithm
algorithms = [
    SVD(n_epochs = 1, n_factors = 20), 
    SVD(n_epochs = 2, n_factors = 20), 
    SVD(n_epochs = 5) 
]
algorithm = algorithms[0]

# Perform cross validation
results = cross_validate(algorithm, data, measures=['RMSE'], cv=5, verbose=True)

# Get results & append algorithm name
tmp = pd.DataFrame.from_dict(results).mean(axis=0)
tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]]))
benchmark.append(tmp)


Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9032  0.9054  0.9042  0.9047  0.9044  0.9044  0.0007  
Fit time          1.36    2.00    2.03    1.99    1.99    1.87    0.26    
Test time         2.25    2.19    2.29    2.25    2.89    2.37    0.26    


In [58]:
from surprise.model_selection import train_test_split
from surprise import accuracy
trainset, testset = train_test_split(data, test_size=.25)
algorithm.fit(trainset)
predictions = algorithm.test(testset)
accuracy.rmse(predictions)

RMSE: 0.9073


0.9073485600371427

In [59]:
def get_collabration_based_recommendations(userId, movieId):
    return algorithm.predict(userId, movieId).est

In [60]:
def get_recommendations(userId):
    content_based = get_content_based_recommendation_for_user(userId)
    content_based = content_based[['title','movieId']]
    
    predicted_rating = []
    for i in range(len(content_based)):
        ratings = get_collabration_based_recommendations(userId, int(content_based.iloc[i].movieId))
        predicted_rating.append(ratings)
    content_based['predicted_rating'] = predicted_rating
    content_based = content_based.sort_values(by=['predicted_rating'], ascending=False)

    return content_based[:10]



In [61]:
get_recommendations(1)

,title,movieId,predicted_rating
0,small_change,1660,3.579189
1,the_debt,52666,3.579189
24,when_harry_met_sally_,639,3.579189
23,bed_and_board,258,3.579189
22,the_double_life_of_veronique,1600,3.579189
21,the_last_metro,1716,3.579189
20,the_lady_vanishes,940,3.579189
19,frenzy,573,3.579189
18,blackmail,543,3.579189
17,three_colors_white,109,3.579189
